#**Homework 2 - Classification**

## **goal**

Binary classification is one of the most fundamental problem in machine learning. In this tutorial, you are going to build linear binary classifiers to predict whether the income of an indivisual exceeds 50,000 or not. We presented a discriminative and a generative approaches, the logistic regression(LR) and the linear discriminant anaysis(LDA). You are encouraged to compare the differences between the two, or explore more methodologies. Although you can finish this tutorial by simpliy copying and pasting the codes, we strongly recommend you to understand the mathematical formulation first to get more insight into the two algorithms. Please find [here](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Logistic%20Regression%20(v3).pdf) and [here](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Classification%20(v3).pdf) for more detailed information about the two algorithms.

二元分類是機器學習中最基礎的問題之一，在這份教學中，你將學會如何實作一個線性二元分類器，來根據人們的個人資料，判斷其年收入是否高於 50,000 美元。我們將以兩種方法: logistic regression 與 generative model，來達成以上目的，你可以嘗試了解、分析兩者的設計理念及差別。針對這兩個演算法的理論基礎，可以參考李宏毅老師的教學投影片 [logistic regression](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Logistic%20Regression%20(v3).pdf) 與 [generative model](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Classification%20(v3).pdf)。

若有任何問題，歡迎來信至助教信箱 ntu-ml-2020spring-ta@googlegroups.com

 ----- strong baseline -----   0.89102

----- simple baseline -----   0.88675

#Dataset

This dataset is obtained by removing unnecessary attributes and balancing the ratio between positively and negatively labeled data in the [**Census-Income (KDD) Data Set**](https://archive.ics.uci.edu/ml/datasets/Census-Income+(KDD)), which can be found in [**UCI Machine Learning Repository**](https://archive.ics.uci.edu/ml/index.php). Only preprocessed and one-hot encoded data (i.e. *X_train*,  *Y_train* and *X_test*) will be used in this tutorial. Raw data (i.e. *train.csv* and *test.csv*) are provided to you in case you are interested in it.

這個資料集是由 [**UCI Machine Learning Repository**](https://archive.ics.uci.edu/ml/index.php) 的 [**Census-Income (KDD) Data Set**](https://archive.ics.uci.edu/ml/datasets/Census-Income+(KDD)) 經過一些處理而得來。為了方便訓練，我們移除了一些不必要的資訊，並且稍微平衡了正負兩種標記的比例。事實上在訓練過程中，只有 X_train、Y_train 和 X_test 這三個經過處理的檔案會被使用到，train.csv 和 test.csv 這兩個原始資料檔則可以提供你一些額外的資訊。

In [ ]:
!gdown --id '1KSFIRh0-_Vr7SdiSCZP1ItV7bXPxMD92' --output data.tar.gz # download the dataset as data.tar.gz
!tar -zxvf data.tar.gz # 解壓
!ls

#Logistic Regression

In this section we will introduce logistic regression first. We only present how to implement it here, while mathematical formulation and analysis will be omitted. You can find more theoretical detail in [Prof. Lee's lecture](https://www.youtube.com/watch?v=hSXFuypLukA).

首先我們會實作 logistic regression，針對理論細節說明請參考[李宏毅老師的教學影片](https://www.youtube.com/watch?v=hSXFuypLukA)

###Preparing Data

Load and normalize data, and then split training data into training set and development set.

下載資料，並且對每個屬性做正規化，處理過後再將其切分為訓練集與發展集。

In [ ]:
import numpy as np

np.random.seed(0)
X_train_fpath = './data/X_train' # set the file path
Y_train_fpath = './data/Y_train'
X_test_fpath = './data/X_test'
output_fpath = './output_{}.csv'

# Parse csv files to numpy array
with open(X_train_fpath) as f: # open the file
    next(f) # next() 返回迭代器的下一個項目，平時會與 iter() 合用。
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:
        specified_column = np.arange(X.shape[1]) # 產生一個 column 為 X.shape[1] 長度的array
    if train:
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1) 
        # reshape 特殊用法：mat (or array).reshape(c, -1)
        # 必須是矩陣或數組格式，才能使用 .reshape(c, -1) 函数 
        # 表示重組此矩陣或是數組，以 c 行 d 列的形式表示（-1的作用就在此，自動計算 d：d = 數組或矩陣的所有元素個數 /c, d 必須為整數，不然報錯）
        # reshape(-1, e) 即列數固定，行數重新計算

        # mean : axis = 0：壓縮行，對每一列求平均值，返回 1* n 矩陣
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1)

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8) # normalize
     
    return X, X_mean, X_std

def _train_dev_split(X, Y, dev_ratio = 0.25):
    # This function spilts data into training set and development set.
    train_size = int(len(X) * (1 - dev_ratio))
    return X[:train_size], Y[:train_size], X[train_size:], Y[train_size:]

# Normalize training and testing data
X_train, X_mean, X_std = _normalize(X_train, train = True) # 決定是否標準化
X_test, _, _= _normalize(X_test, train = False, specified_column = None, X_mean = X_mean, X_std = X_std)
    
# Split data into training set and development set
# training set：訓練集是用來訓練模型，占了數據的絕大部分。
# development set：用來對模型測試的資料集，通過測試來不斷優化模型。
# test set：訓練結束後，對訓練出的模型進行最後一次評估的數據集。
dev_ratio = 0.1 # 可以自行調配 train, dev 數量
X_train, Y_train, X_dev, Y_dev = _train_dev_split(X_train, Y_train, dev_ratio = dev_ratio)

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]
print('Size of training set: {}'.format(train_size))
print('Size of development set: {}'.format(dev_size))
print('Size of testing set: {}'.format(test_size))
print('Dimension of data: {}'.format(data_dim))


###Some Useful Functions

Some functions that will be repeatedly used when iteratively updating the parameters.

這幾個函數可能會在訓練迴圈中被重複使用到。

In [ ]:
def _shuffle(X, Y):
    # This function shuffles two equal-length list/array, X and Y, together.
    randomize = np.arange(len(X)) # 產生元素從 0 開始至 len(x)-1 的 1*len(x) array (公差 = 1)
    np.random.shuffle(randomize) # np.random.shuffle(): Modify a sequence in-place by shuffling its contents
    return (X[randomize], Y[randomize])

def _sigmoid(z):
    # Sigmoid function can be used to calculate probability.
    # To avoid overflow, minimum/maximum output value is set.
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))

def _f(X, w, b):
    # This is the logistic regression function, parameterized by w and b
    #
    # Arguements:
    #     X: input data, shape = [batch_size, data_dimension]
    #     w: weight vector, shape = [data_dimension, ]
    #     b: bias, scalar
    # Output:
    #     predicted probability of each row of X being positively labeled, shape = [batch_size, ]
    return _sigmoid(np.matmul(X, w) + b) # np.matmul(): 矩陣相乘

def _predict(X, w, b):
    # This function returns a truth value prediction for each row of X 
    # by rounding the result of logistic regression function.
    return np.round(_f(X, w, b)).astype(np.int) # np.round: 對傳入的數據進行四捨五入，如果ngigits不傳，默認是0（就是說保留整數部分）
    
def _accuracy(Y_pred, Y_label):
    # This function calculates prediction accuracy
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc

### Functions about gradient and loss

Please refers to [Prof. Lee's lecture slides](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Logistic%20Regression%20(v3).pdf)(p.12) for the formula of gradient and loss computation.

請參考[李宏毅老師上課投影片](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Logistic%20Regression%20(v3).pdf)第 12 頁的梯度及損失函數計算公式。

In [28]:
def _cross_entropy_loss(y_pred, Y_label):
    # This function computes the cross entropy.
    #
    # Arguements:
    #     y_pred: probabilistic predictions, float vector
    #     Y_label: ground truth labels, bool vector
    # Output:
    #     cross entropy, scalar
    cross_entropy = -np.dot(Y_label, np.log(y_pred)) - np.dot((1 - Y_label), np.log(1 - y_pred)) # np.log 下沒寫默認底數是自然對數 
    return cross_entropy

def _gradient(X, Y_label, w, b):
    # This function computes the gradient of cross entropy loss with respect to weight w and bias b.
    y_pred = _f(X, w, b)
    pred_error = Y_label - y_pred
    w_grad = -np.sum(pred_error * X.T, 1)
    b_grad = -np.sum(pred_error)
    return w_grad, b_grad

def _gradient_L1(X, Y_label, w, b, lambda_L1):
    # This function computes the gradient of cross entropy loss with respect to weight w and bias b.
    y_pred = _f(X, w, b)
    pred_error = Y_label - y_pred
    w_grad = -np.sum(pred_error * X.T, 1) + lambda_L1*(w/(abs(w)))
    b_grad = -np.sum(pred_error)
    return w_grad, b_grad

def _gradient_L2(X, Y_label, w, b, lambda_L2):
    # This function computes the gradient of cross entropy loss with respect to weight w and bias b.
    y_pred = _f(X, w, b)
    pred_error = Y_label - y_pred
    w_grad = -np.sum(pred_error * X.T, 1) + 2*lambda_L2*w
    b_grad = -np.sum(pred_error)
    return w_grad, b_grad

### Training

Everything is prepared, let's start training! 

Mini-batch gradient descent is used here, in which training data are split into several mini-batches and each batch is fed into the model sequentially for losses and gradients computation. Weights and bias are updated on a mini-batch basis.

Once we have gone through the whole training set,  the data have to be re-shuffled and mini-batch gradient desent has to be run on it again. We repeat such process until max number of iterations is reached.

我們使用小批次梯度下降法來訓練。訓練資料被分為許多小批次，針對每一個小批次，我們分別計算其梯度以及損失，並根據該批次來更新模型的參數。當一次迴圈完成，也就是整個訓練集的所有小批次都被使用過一次以後，我們將所有訓練資料打散並且重新分成新的小批次，進行下一個迴圈，直到事先設定的迴圈數量達成為止。

In [ ]:
# Zero initialization for weights ans bias
# w = np.zeros((data_dim,)) # 產生 data_dim*1 的 zero array
# b = np.zeros((1,)) # 產生 1*1 的 zero array

# 這裡使用 one array，因為當 L1_regularization 時，一開始 w 無法在 0 的位置產生梯度，因此初始化我們採用 one array
w = np.ones((data_dim,)) # 產生 data_dim*1 的 one array
b = np.ones((1,)) # 產生 1*1 的 one array

# Some parameters for training # 可調看看不同參數   
max_iter = 550
# batchsize：中文翻譯為批大小（批尺寸）。簡單點說，批量大小將決定我們一次訓練的樣本數目。batch_size將影響到模型的優化程度和速度。
# 為什麼需要有 Batch_Size： batchsize 的正確選擇是為了在記憶體效率和記憶體容量之間尋找最佳平衡。
# 相對於正常資料集，如果 Batch_Size 過小，訓練資料就會非常難收斂，從而導致 underfitting。
# 增大 Batch_Size, 相對處理速度加快。
# 增大 Batch_Size, 所需記憶體容量增加（epoch的次數需要增加以達到最好的結果）
# 這裡我們發現上面兩個矛盾的問題，因為當 epoch 增加以後同樣也會導致耗時增加從而速度下降。因此我們需要尋找最好的 Batch_Size。
# 再次重申：Batch_Size 的正確選擇是為了在記憶體效率和記憶體容量之間尋找最佳平衡。
batch_size = 16
learning_rate = 0.08

# Keep the loss and accuracy at every iteration for plotting
train_loss = []
dev_loss = []
train_acc = []
dev_acc = []
lambda_list = [0.01, 0.1, 0, 1, 10, 100]
train_lambda = []
dev_lambda = []

# Calcuate the number of parameter updates
step = 1
############ 計算 training 時間
# -*- coding: utf-8 -*-
import time
tStart = time.time() # 計時測量 training 開始
# Iterative training
for l in lambda_list: # 運用不同 lambda
    train_loss = []
    dev_loss = []
    train_acc = []
    dev_acc = []
    w = np.ones((data_dim,)) # 產生 data_dim*1 的 one array
    b = np.ones((1,)) # 產生 1*1 的 one array
    for epoch in range(max_iter):
        # Random shuffle at the begging of each epoch
        X_train, Y_train = _shuffle(X_train, Y_train)
            
        # Mini-batch training
        for idx in range(int(np.floor(train_size / batch_size))):
            X = X_train[idx*batch_size:(idx+1)*batch_size]
            Y = Y_train[idx*batch_size:(idx+1)*batch_size]

            # Compute the gradient
            w_grad, b_grad = _gradient(X, Y, w, b)
            # w_grad, b_grad =  _gradient_L1(X, Y, w, b, l)
            # w_grad, b_grad =  _gradient_L2(X, Y, w, b, l)
                
            # gradient descent update
            # learning rate decay with time
            w = w - learning_rate/np.sqrt(step) * w_grad
            b = b - learning_rate/np.sqrt(step) * b_grad

            step = step + 1
                
        # Compute loss and accuracy of training set and development set
        y_train_pred = _f(X_train, w, b)
        Y_train_pred = np.round(y_train_pred)
        train_acc.append(_accuracy(Y_train_pred, Y_train))
        train_loss.append(_cross_entropy_loss(y_train_pred, Y_train) / train_size)

        y_dev_pred = _f(X_dev, w, b)
        Y_dev_pred = np.round(y_dev_pred)
        dev_acc.append(_accuracy(Y_dev_pred, Y_dev))
        dev_loss.append(_cross_entropy_loss(y_dev_pred, Y_dev) / dev_size)
        print('epoch =', epoch, ' ,train_acc =', _accuracy(Y_train_pred, Y_train), ' ,dev_acc =', _accuracy(Y_dev_pred, Y_dev))
    
    train_lambda.append(train_acc)
    dev_lambda.append(dev_acc)
    # end of 要測量的training
    tEnd = time.time() #計時結束
    #列印結果
    print('Training time is '+str(tEnd - tStart)) #顯示 training 時間
    print('Training loss: {}'.format(train_loss[-1]))
    print('Development loss: {}'.format(dev_loss[-1]))
    print('Training accuracy: {}'.format(train_acc[-1]))
    print('Development accuracy: {}'.format(dev_acc[-1]))

### Plotting Loss and accuracy curve

In [ ]:
import matplotlib.pyplot as plt

# Loss curve
plt.plot(train_loss)
plt.plot(dev_loss)
plt.title('Loss')
plt.legend(['train', 'dev'])
plt.savefig('loss.png')
plt.show()

# Accuracy curve
plt.plot(train_acc)
plt.plot(dev_acc)
plt.title('Logistic_Accuracy')
plt.legend(['train', 'dev'])
plt.savefig('acc.png')
plt.show()

# # show the L1 regularization image
# # L1_train Accuracy curve
# for i in range(6):
#     plt.plot(train_lambda[i][:200])
# plt.title('L1 Training Accuracy')
# plt.legend(['Lambda: 0.01', 'Lambda: 0.1', 'Lambda: 0', 'Lambda: 1', 'Lambda: 10', 'Lambda: 100'])
# plt.savefig('acc.png')
# plt.show()

# # L1_dev Accuracy curve
# for i in range(6):
#     plt.plot(dev_lambda[i][:200])
# plt.title('L1 Dev Accuracy')
# plt.legend(['Lambda: 0.01', 'Lambda: 0.1', 'Lambda: 0', 'Lambda: 1', 'Lambda: 10', 'Lambda: 100'])
# plt.savefig('acc.png')
# plt.show()

# show the L2 regularization image
# # L2_train Accuracy curve 
# for i in range(6):
#     plt.plot(train_lambda[i][:200])
# plt.title('L2 Training Accuracy')
# plt.legend(['Lambda: 0.01', 'Lambda: 0.1', 'Lambda: 0', 'Lambda: 1', 'Lambda: 10', 'Lambda: 100'])
# plt.savefig('acc.png')
# plt.show()

# # L2_dev Accuracy curve 
# for i in range(6):
#     plt.plot(dev_lambda[i][:200])
# plt.title('L2 Dev Accuracy')
# plt.legend(['Lambda: 0.01', 'Lambda: 0.1', 'Lambda: 0', 'Lambda: 1', 'Lambda: 10', 'Lambda: 100'])
# plt.savefig('acc.png')
# plt.show()

###Predicting testing labels

Predictions are saved to *output_logistic.csv*.

預測測試集的資料標籤並且存在 *output_logistic.csv* 中。

In [ ]:
# Predict testing labels
predictions = _predict(X_test, w, b)
with open(output_fpath.format('logistic'), 'w') as f:
    f.write('id,label\n')
    for i, label in  enumerate(predictions):
        f.write('{},{}\n'.format(i, label))
# Print out the most significant weights
ind = np.argsort(np.abs(w))[::-1] # np.argsort: 返回數組值從小到大的索引值，經過 [::-1] 順序將從大至小
with open(X_test_fpath) as f:
    content = f.readline().strip('\n').split(',')
features = np.array(content)
for i in ind[0:10]:
    print(features[i], w[i])

#存到本機端
# from google.colab import files
# files.download('output_logistic.csv')

# Porbabilistic generative model

In this section we will discuss a generative approach to binary classification. Again, we will not go through the formulation detailedly. Please find [Prof. Lee's lecture](https://www.youtube.com/watch?v=fZAZUYEeIMg) if you are interested in it.

接者我們將實作基於 generative model 的二元分類器，理論細節請參考[李宏毅老師的教學影片](https://www.youtube.com/watch?v=fZAZUYEeIMg)。

### Preparing Data

Training and testing data is loaded and normalized as in logistic regression. However, since LDA is a deterministic algorithm, there is no need to build a development set.

訓練集與測試集的處理方法跟 logistic regression 一模一樣，然而因為 generative model 有可解析的最佳解，因此不必使用到 development set。

In [45]:
# Parse csv files to numpy array
with open(X_train_fpath) as f:
    next(f)
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

# Normalize training and testing data
X_train, X_mean, X_std = _normalize(X_train, train = True)
X_test, _, _= _normalize(X_test, train = False, specified_column = None, X_mean = X_mean, X_std = X_std)

### Mean and Covariance

In generative model, in-class mean and covariance are needed.

在 generative model 中，我們需要分別計算兩個類別內的資料平均與共變異。

In [46]:
# Compute in-class mean
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])

mean_0 = np.mean(X_train_0, axis = 0)
mean_1 = np.mean(X_train_1, axis = 0)  

# Compute in-class covariance
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0]
for x in X_train_1:
    # cov_1 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0] #試試看用相同的 cov 此時結果不是理想 (可參考老師 PPT)
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]

# Shared covariance is taken as a weighted average of individual in-class covariance.
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])

### Computing weights and bias

Directly compute weights and bias from in-class mean and shared variance. [Prof. Lee's lecture slides](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Classification%20(v3).pdf)(p.33) gives a concise explanation.

權重矩陣與偏差向量可以直接被計算出來，算法可以參考[李宏毅老師教學投影片](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/Classification%20(v3).pdf)第 33 頁。

In [ ]:
# Compute inverse of covariance matrix.
# Since covariance matrix may be nearly singular, np.linalg.inv() may give a large numerical error.
# Via SVD decomposition, one can get matrix inverse efficiently and accurately.
u, s, v = np.linalg.svd(cov, full_matrices=False)

# 函數：np.linalg.svd(a, full_matrices = 1, compute_uv = 1)。
# 參數介紹：
# a是一個形如(M,N)矩陣
# full_matrices的取值是為 0 或者 1，默認值為 1，這時 u 的大小為 (M,M)，v 的大小為 (N,N)。否則 u 的大小為 (M,K)，v 的大小為 (K,N)，K = min(M,N)。
# compute_uv 的取值是為 0 或者 1 ，默認值為 1，表示計算 u, s, v。為 0 的時候只計算 s。

# 返回值：
# 總共有三個返回值 u,s,v
# u 大小為 (M,M)，s大小為 (M,N)，v 大小為 (N,N)。
# A = u*s*v
# 其中 s 是對矩陣 a 的奇異值分解。 s 除了對角元素不為 0，其他元素都為 0，並且對角元素從大到小排列。 
# s 中有 n 個奇異值，一般排在後面的比較接近 0，所以僅保留比較大的 r 個奇異值。

inv = np.matmul(v.T * 1 / s, u.T)

# Directly compute weights and bias
w = np.dot(inv, mean_0 - mean_1)
b =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0]) 

# Compute accuracy on training set
Y_train_pred = 1 - _predict(X_train, w, b)
print('Training accuracy: {}'.format(_accuracy(Y_train_pred, Y_train)))

###Predicting testing labels

Predictions are saved to *output_generative.csv*.

預測測試集的資料標籤並且存在 *output_generative.csv* 中。

In [ ]:
# Predict testing labels
predictions = 1 - _predict(X_test, w, b)
with open(output_fpath.format('generative'), 'w') as f:
    f.write('id,label\n')
    for i, label in  enumerate(predictions):
        f.write('{},{}\n'.format(i, label))

# Print out the most significant weights
ind = np.argsort(np.abs(w))[::-1]
with open(X_test_fpath) as f:
    content = f.readline().strip('\n').split(',')
features = np.array(content)
for i in ind[0:10]:
    print(features[i], w[i])
